In [ ]:
# Para el tratamiento de datos:
import pandas as pd
import numpy as np
from datetime import datetime
import unicodedata
import os
pd.set_option('display.max_columns', None) #Para ver todas las columnas

In [ ]:
dfh = pd.read_csv("Primary-data/Highest_victim_count.csv")
df5 = pd.read_csv("Primary-data/15_to_30_victim_count.csv")
df4 = pd.read_csv("Primary-data/5_to_14_victim_count.csv")
dfl = pd.read_csv("Primary-data/Lessthan_5_victim_count.csv")

# Paso 1:

## Comenzamos con un EDA inicial de los diferentes datasets

In [ ]:
dfh.head()  #Mas primeras 5 filas del DataFrame (por defecto, pero puedes especificar un número).

In [ ]:
dfh.tail()  #Las últimas 5 filas del DataFrame (por defecto, pero puedes especificar un número).

In [ ]:
dfh.sample()  #Una fila aleatoria del DataFrame (puedes especificar cuántas filas con `n`).

In [ ]:
dfh.columns  #Los nombres de las columnas.

In [ ]:
dfh.shape #Una tupla con el número de filas y columnas del DataFrame (filas, columnas).

In [ ]:
dfh.info()  #Info del DataFrame: número de filas, columnas y tipos de datos.

In [ ]:
dfh.dtypes #Tipo de datos de cada columna.

In [ ]:
dfh.describe().T #Suma, media, desviación estandar, mínimo/máximo y percentiles (valores numéricos).

In [ ]:
dfh.describe(include = 'object') #Suma, valores únicos... de tipo object.

In [ ]:
#Bucle con el que extraemos datos relevantes de cada columna:
columnas = dfh.columns

for col in columnas:
    print(f"\n🔹'{col}'\n")
    print(f"Tipo de dato: {dfh[col].dtypes}\n")
    print(f"Porcentaje de nulos: {dfh[col].isnull().sum()/dfh.shape[0]*100}\n")
    print(f"Suma de duplicados: {dfh[col].duplicated().sum()}\n")
    print(f"Valores únicos:")
    print(dfh[col].unique())
    print("-" * 100)

In [ ]:
df5.head()  #Mas primeras 5 filas del DataFrame (por defecto, pero puedes especificar un número).

In [ ]:
df5.tail()  #Las últimas 5 filas del DataFrame (por defecto, pero puedes especificar un número).

In [ ]:
df5.sample()  #Una fila aleatoria del DataFrame (puedes especificar cuántas filas con `n`).

In [ ]:
df5.columns  #Los nombres de las columnas.

In [ ]:
df5.shape #Una tupla con el número de filas y columnas del DataFrame (filas, columnas).

In [ ]:
df5.info()  #Info del DataFrame: número de filas, columnas y tipos de datos.

In [ ]:
df5.dtypes #Tipo de datos de cada columna.

In [ ]:
df5.describe().T #Suma, media, desviación estandar, mínimo/máximo y percentiles (valores numéricos).

In [ ]:
df5.describe(include = 'object') #Suma, valores únicos... de tipo object.

In [ ]:
#Bucle con el que extraemos datos relevantes de cada columna:
columnas = df5.columns


for col in columnas:
    print(f"\n🔹'{col}'\n")
    print(f"Tipo de dato: {df5[col].dtypes}\n")
    print(f"Porcentaje de nulos: {df5[col].isnull().sum()/df5.shape[0]*100}\n")
    print(f"Suma de duplicados: {df5[col].duplicated().sum()}\n")
    print(f"Valores únicos:")
    print(df5[col].unique())
    print("-" * 100)

In [ ]:
df4.head()  #Mas primeras 5 filas del DataFrame (por defecto, pero puedes especificar un número).

In [ ]:
df4.tail()  #Las últimas 5 filas del DataFrame (por defecto, pero puedes especificar un número).

In [ ]:
df4.sample()  #Una fila aleatoria del DataFrame (puedes especificar cuántas filas con `n`).

In [ ]:
df4.columns  #Los nombres de las columnas.

In [ ]:
df4.shape #Una tupla con el número de filas y columnas del DataFrame (filas, columnas).

In [ ]:
df4.info()  #Info del DataFrame: número de filas, columnas y tipos de datos.

In [ ]:
df4.dtypes #Tipo de datos de cada columna.

In [ ]:
df4.describe() #Suma, media, desviación estandar, mínimo/máximo y percentiles (valores numéricos).

In [ ]:
df4.describe(include = 'object') #Suma, valores únicos... de tipo object.

In [ ]:
#Bucle con el que extraemos datos relevantes de cada columna:
columnas = df4.columns

for col in columnas:
    print(f"\n🔹'{col}'\n")
    print(f"Tipo de dato: {df4[col].dtypes}\n")
    print(f"Porcentaje de nulos: {df4[col].isnull().sum()/df4.shape[0]*100}\n")
    print(f"Suma de duplicados: {df4[col].duplicated().sum()}\n")
    print(f"Valores únicos:")
    print(df4[col].unique())
    print("-" * 100)

In [ ]:
dfl.head()  #Mas primeras 5 filas del DataFrame (por defecto, pero puedes especificar un número).

In [ ]:
dfl.tail()  #Las últimas 5 filas del DataFrame (por defecto, pero puedes especificar un número).

In [ ]:
dfl.sample()  #Una fila aleatoria del DataFrame (puedes especificar cuántas filas con `n`).

In [ ]:
dfl.columns  #Los nombres de las columnas.

In [ ]:
dfl.shape #Una tupla con el número de filas y columnas del DataFrame (filas, columnas).

In [ ]:
dfl.info()  #Info del DataFrame: número de filas, columnas y tipos de datos.

In [ ]:
dfl.dtypes #Tipo de datos de cada columna.

In [ ]:
dfl.describe() #Suma, media, desviación estandar, mínimo/máximo y percentiles (valores numéricos).

In [ ]:
dfl.describe(include = 'object') #Suma, valores únicos... de tipo object.

In [ ]:
#Bucle con el que extraemos datos relevantes de cada columna:
columnas = dfl.columns

for col in columnas:
    print(f"\n🔹'{col}'\n")
    print(f"Tipo de dato: {dfl[col].dtypes}\n")
    print(f"Porcentaje de nulos: {dfl[col].isnull().sum()/dfl.shape[0]*100}\n")
    print(f"Suma de duplicados: {dfl[col].duplicated().sum()}\n")
    print(f"Valores únicos:")
    print(dfl[col].unique())
    print("-" * 100)

## A partir de aqui hacemos varias comprovaciones para asegurarnos que los datasets son iguales y se puede concatenar

In [ ]:
print(dfh.shape)
dfh['Name'].nunique()

In [ ]:
print(df5.shape)
df5['Name'].nunique()

In [ ]:
print(df4.shape)
df4['Name'].nunique()

In [ ]:
print(dfl.shape)
dfl['Name'].nunique()

# Paso 2:

## Pasamos a la limpieza y normalizacion de los datos

In [ ]:
# Concatenar los df:
dfc = pd.concat([dfh, df5, df4, dfl], ignore_index=True)

In [ ]:
# Revisar duplicados
dfc.duplicated().sum()

In [ ]:
# Cambiar los espacios por "_":
dfc.columns #Me da todas las columnas

# Diccionario vacío:
col_f = {} 

# Itera dfc.columns y crea un diccionario clave : valor con el cambio.
for columna in dfc.columns:
    col_f[columna] = columna.replace(" ", "_")

# Rename:
dfc.rename(columns = col_f, inplace = True)

In [ ]:
dfc.columns

In [ ]:
dfc['Years_active'].to_frame().T

In [ ]:
# Dividir 'Years_active' en 'Start Stop' y modificar la columna 'Years_active' (total años en activo)
# Crear la columna 'Start_year' a partir de 'Years_active': Eliminando el 'to' y lo que va después.
dfc['Start_year'] = dfc['Years_active'].str.split(" to ").str[0]

In [ ]:
dfc['Start_year'].unique()

In [ ]:
# Las columnas son una caca y tuve que preguntarle a chatgpt qué hacer, después de provar mil cosas me dió este código:
# Extraer los cuatro primero dígitos de la cadena:
dfc['Start_year'] = dfc['Start_year'].str.extract(r'(\d{4})')

# Convertir la columna a numérico:
dfc['Start_year'] = pd.to_numeric(dfc['Start_year'], errors='coerce')

In [ ]:
# Revisar que todo ok:
dfc['Start_year'].to_frame().T

In [ ]:
# Para 'End_year' también era un lío así que me ayudó de nuevo chatgpt.
# Extraer todos los números de 4 dígitos en una lista para cada registro y tomar solamente el último, si no hay devuelve nulo.
dfc['End_year'] = dfc['Years_active'].str.findall(r'(\d{4})').apply(lambda x: x[-1] if x else np.nan)

# Convertir la columna a numérico
dfc['End_year'] = pd.to_numeric(dfc['End_year'], errors='coerce')

In [ ]:
# Revisar again que todo ok:
dfc['End_year'].to_frame().T

In [ ]:
# Comparar las dos columnas:
dfc['Start_year'].to_frame().T

Al comparar los datos de 'Years_active', 'Start_year' y 'End_year' hay que cambiar ciertas entradas en 'End_year':
- A 1959 en_ 92, 108 y 27. Ya que estuvieron activos en 1950s.
- A 1999 en: 58 y 242. Ya que estuvieron activos en 1990s.
- A 2009 en: 224 y 225. Ya que estuvieron activos en 2000s.
- En el id 198 indica "to present", ¿asumimos que la fecha es 2018 ya que es el último año de actividad?

In [ ]:
# Ver cuál es el último año:
unique_values = np.sort(dfc['End_year'].unique())[::-1]
print(unique_values)

In [ ]:
#Cambios en 'End_year'.
# A 1959 en_ 92, 108 y 27. Ya que estuvieron activos en 1950s.
dfc.loc[[92, 108, 27], 'End_year'] = 1959

# A 1999 en: 58 y 242. Ya que estuvieron activos en 1990s.
dfc.loc[[58, 242], 'End_year'] = 1999

# A 2009 en: 24 y 25. Ya que estuvieron activos en 2000s.
dfc.loc[[224, 225], 'End_year'] = 2009

# En el id 198 indica "to present", ¿asumimos que la fecha es 2018 ya que es el último año de actividad?
dfc.loc[[198], 'End_year'] = 2018

In [ ]:
# Columna nueva con años en activo, nuevo nombre para que no se solape:
# Añadimos el +1 para que los que llevan solo un año en activo no se quede 0 y para que el primer año también cuente en el resto.
dfc['Años_activo'] = dfc['End_year'] - dfc['Start_year'] + 1

In [ ]:
dfc['Años_activo'].unique()

In [ ]:
# Eliminar 'Years_active' para después cambiar 'Años_activo':
dfc.drop('Years_active', axis=1, inplace=True)

In [ ]:
# Cambio de nombre:
dfc.rename(columns = {'Años_activo': 'Years_active'}, inplace = True)

Vamos ahora con 'Proven_victims'

In [ ]:
dfc['Proven_victims'].unique()

In [ ]:
# Hago ua función para cambiar los valores que "no sirven" porque intentando hacerlo con código la liaba siempre.
def intervalos(Proven_victims):
    if Proven_victims == '11–16':
        return "11"
    elif Proven_victims == "9–13":
        return "9"
    elif Proven_victims == "8–15":
        return "8"
    elif Proven_victims == "8–11+":
        return "8"
    else:
        return Proven_victims

In [ ]:
# Aplica la función:
dfc['Proven_victims'] = dfc['Proven_victims'].apply(intervalos)

In [ ]:
# Reviso:
dfc['Proven_victims'].unique()

In [ ]:
# Cambio a int:
dfc['Proven_victims'] = dfc['Proven_victims'].astype(int)

In [ ]:
# Reviso again:
dfc['Proven_victims'].unique()

Nos encontramos que en los registros de la columna country aparecen listas de paises separados por saltos de fila. Tableau no puede interpretar estos datos correctamente 
por lo que trás hablar con la PO tomamos la decision de hacer un .explode por filas y quedarnos con la primera.

In [ ]:
print(dfc['Country'].unique())

In [ ]:
#Separamos los países por salto de línea, explotamos filas y limpiamos espacios delante y detrás
dfc['Country'] = dfc['Country'].str.split(r'\r\n')
dfc = dfc.explode('Country')
dfc['Country'] = dfc['Country'].str.strip()
dfc = dfc.drop_duplicates(subset='Name', keep='first').reset_index(drop=True)


In [ ]:
duplicates = dfc.duplicated(subset='Name').sum()
print(f'Duplicados encontrados: {duplicates}')


In [ ]:
# comprobamos que se ha ejecutado correctamente
print(sorted(dfc['Country'].unique()))
print(dfc['Country'].isnull().sum())
dfc['Country'].value_counts()



In [ ]:
dfc.head(5)

Vemos que tras la limpieza nuestro dataset son pocos registros pasa analizar y añadimos dos csv más para completar

In [ ]:
df_victims = pd.read_csv("Primary-data/serial_killers_by_victims.csv", sep=";")
df_profile = pd.read_csv("Primary-data/profile_killers.csv", sep=";")

In [ ]:
df_profile.info()
df_victims.info()
dfc.info()

In [ ]:
print("Columnas en df_victims:")
print(df_victims.columns)

print("Columnas en df_profile:")
print(df_profile.columns)


In [ ]:
df_new_killers = df_victims.merge(df_profile, on='Name', how='outer')

In [ ]:
# Cambiar los espacios por "_":
df_new_killers.columns

col_f = {} 

# Itera dfc.columns y crea un diccionario clave : valor con el cambio.
for columna in df_new_killers.columns:
    col_f[columna] = columna.replace(" ", "_")

# Rename:
df_new_killers.rename(columns = col_f, inplace = True)

#vemos los cambios en columnas
print(df_new_killers.columns)

In [ ]:
df_new_killers.info()

In [ ]:
#VEr uplicados por nombre
duplicados = df_new_killers[df_new_killers.duplicated(subset='Name', keep=False)]
print(f"Duplicados por 'Name': {duplicados.shape[0]}")
if not duplicados.empty:
    print(duplicados[['Name', 'Country', 'Start_year', 'End_year']])

#Ver nulos por columna
print("Valores nulos por columna:")
print(df_new_killers.isnull().sum())

#Ver si End_year es menor que Start_year
errores_fecha = df_new_killers[df_new_killers['End_year'] < df_new_killers['Start_year']]
print(f"End_year < Start_year: {errores_fecha.shape[0]}")
if not errores_fecha.empty:
    print(errores_fecha[['Name', 'Start_year', 'End_year']])

#Ver si el número de victimas es negativo o 0
errores_victimas = df_new_killers[df_new_killers['Proven_victims'] <= 0]
print(f"Proven_victims <= 0: {errores_victimas.shape[0]}")
if not errores_victimas.empty:
    print(errores_victimas[['Name', 'Proven_victims']])

#Ver si hay registros sin nombre
sin_nombre = df_new_killers['Name'].isnull().sum()
print(f"Registros sin nombre: {sin_nombre}")


In [ ]:
#VER columnas disponibles en ambos
print(dfc.columns)
print(df_new_killers.columns)

#Hacemos merge con outer n para mantener todos los asesinos
df_final = pd.merge(dfc, df_new_killers, on='Name', how='outer', suffixes=('_kaggle', '_extra'))

#Ver si hay coincidencias filtrando por la columna name
matches = dfc['Name'].isin(df_new_killers['Name']).sum()
print(f"Coincidencias por 'Name': {matches} de {len(dfc)}")


In [ ]:
df_final.info()

In [ ]:
df_final.sample(10)

In [ ]:
#Ya sabemos que Name es identificador único, ahora vamos a ver si hay duplicados por nombre.
duplicados = df_final[df_final.duplicated(subset='Name', keep=False)]

#En el caso de que haya, quiero ver los nombres que estan duplicados y cuantas veces se repite
nombres_duplicados = duplicados['Name'].value_counts()
print(f"Total asesinos con duplicados: {nombres_duplicados.shape[0]}")
print("Nombres duplicados (con cantidad de repeticiones):")
print(nombres_duplicados)


In [ ]:
#Vemos las columnas comunes que tienen sufijos _kaggle y _extra
columnas_kaggle = [col for col in df_final.columns if col.endswith('_kaggle')]
columnas_extra = [col for col in df_final.columns if col.endswith('_extra')]

columnas_comunes = []
for col_k in columnas_kaggle:
    base_name = col_k.replace('_kaggle', '')
    col_e = base_name + '_extra'
    if col_e in columnas_extra:
        columnas_comunes.append(base_name)

#Analizar ambas direcciones
print("Aportes de datos entre _kaggle y _extra:")
for col in columnas_comunes:
    col_kaggle = f"{col}_kaggle"
    col_extra = f"{col}_extra"

    aporta_kaggle = df_final[col_extra].isnull() & df_final[col_kaggle].notnull()
    aporta_extra = df_final[col_kaggle].isnull() & df_final[col_extra].notnull()

    print(f" {col}")
    print(f"_kaggle aporta en: {aporta_kaggle.sum()} registros donde _extra está vacío")
    print(f"_extra aporta en: {aporta_extra.sum()} registros donde _kaggle está vacío")


In [ ]:
#consolidamos las columnas rellenando los datos faltantes de una columna y otra y eliminamos
columnas_a_consolidar = ['Country', 'Proven_victims', 'Possible_victims', 'Notes', 'Start_year', 'End_year']

for col in columnas_a_consolidar:
    df_final[col] = df_final[f"{col}_extra"].combine_first(df_final[f"{col}_kaggle"])
    df_final.drop(columns=[f"{col}_kaggle", f"{col}_extra"], inplace=True)


In [ ]:
#calculamos los registros faltates de Years_active 
df_final['Years_active'] = df_final['End_year'] - df_final['Start_year']
#y pasamos posibles victimas a numerico
df_final['Possible_victims'] = pd.to_numeric(df_final['Possible_victims'], errors='coerce')

In [ ]:
#Como ya sabemos que Name es un id unico vamos a filtrar los asesinos de los que solo tenemos esa informacion para poder eliminarlos.
solo_nombre = df_final.drop(columns='Name').isnull().all(axis=1)
df_final = df_final[~solo_nombre].reset_index(drop=True)
print(f"Se eliminaron {solo_nombre.sum()} registros con solo el nombre.")



In [ ]:
#La columna country es importante, vamos a pasar a limpiarla y a procesar los nulos.
#podemos usar Born_location para imputar los nulos de country
filtro = df_final['Country'].isnull() & df_final['Born_Location'].notnull()
#Sacamos el país que es la ultima parte de string
df_final.loc[filtro, 'Country'] = df_final.loc[filtro, 'Born_Location'].apply(lambda x: x.split(',')[-1].strip())
# Verificar cuántos se completaron
completados = filtro.sum()
print(f" País estimado desde Born_Location en {completados} registros.")
df_final['Country'].value_counts()
#DE nuevo encontramos paises que esta en una lista, repetimos el codigo que ejecutamos anteriormente para hacer registro unico por asesino y pais y luegos nos quedamos con la primera aparicion, las medas las aliminamos.
#Separamos los países multiples por coma y explotamos las filas
df_final['Country'] = df_final['Country'].str.split(',')
df_final = df_final.explode('Country')
#eliminamos espacios en blanco
df_final['Country'] = df_final['Country'].str.strip()
#dejamos solo una fila por asesino
df_final = df_final.drop_duplicates(subset='Name', keep='first').reset_index(drop=True)
#Pasamos a normalizar los paises, que hay mucho valores inconsistentes. Primero creamos un diccionario que le he pedido a chat gepete que me haga porque esto es perder la vida
normalizacion_paises = {
    # Variantes y abreviaturas
    "USA": "United States",
    "US": "United States",
    "United States of America": "United States",
    "United States of Brazil": "Brazil",
    "Rio de Janeiro": "Brazil",
    "So Paulo": "Brazil",
    "Mxico": "Mexico",
    "Uk": "United Kingdom",
    "UK": "United Kingdom",
    "England": "United Kingdom",
    "Scotland": "United Kingdom",
    "Ontario Canada": "Canada",
    "Territory of Hawaii": "United States",
    "British Hong Kong": "Hong Kong",
    "Imperial State of Iran": "Iran",

    # Históricos o alternativos
    "Allied-occupied Germany": "Germany",
    "West Germany": "Germany",
    "East Germany": "Germany",
    "Nazi Germany": "Germany",
    "German Empire, Germany": "Germany",
    "Soviet Union": "Russia",
    "Soviet Union, Russia": "Russia",
    "Soviet Union, Ukraine": "Ukraine",
    "Soviet Union, Russia, Ukraine": "Russia",
    "Uzbek SSR": "Uzbekistan",
    "UkSSR": "Ukraine",
    "Ottoman Empire": "Turkey",
    "centuryOttoman Empire": "Turkey",
    "Austria-Hungary": "Austria",
    "Austro-Hungarian Empire": "Austria",
    "Czechoslovakia": "Czech Republic",
    "SR Croatia": "Croatia",
    "SR Slovenia": "Slovenia",
    "Yugoslavia": "Serbia",
    "Kingdom of Yugoslavia": "Serbia",
    'Kingdom of Romania': 'Romania',
    "Kingdom of Romania, Yugoslavia, Hungary": "Romania",
    "Cape Colony": "South Africa",

    # Errores o indefinidos
    "TaeiIraq": "Iraq"
}
# Ejecutamos el diccionario 
df_final['Country'] = df_final['Country'].replace(normalizacion_paises)

print("Países únicos después de normalizar:")
print(sorted(df_final['Country'].dropna().astype(str).unique()))



In [ ]:
#continuamos filtrando los asesinos que tiene valores chungos en la columna país
# Filtrar asesinos con Country en los valores problemáticos
valores_chungos = ['s Republic', 'unknown']
df_chungos = df_final[df_final['Country'].isin(valores_chungos)]
display(df_chungos)

In [ ]:
#Imputamos los datos faltantes de estos dos asesinos con datos de wikipedia:
df_final.loc[df_final['Name'] == 'Monster Of Udine', [
    'Nicknames', 'Conviction', 'Criminal_Penalty', 'Country',
    'Proven_victims', 'Possible_victims', 'Start_year', 'End_year', 'Notes']] = [
    'Monster Of Udine',             # Nicknames
    'No conviction',                # Conviction
    'Untried',                      # Criminal_Penalty
    'Italy',                        # Country
    14,                             # Proven_victims
    10,                             # Possible_victims
    1971,                           # Start_year
    1989,                           # End_year
    'The Monster of Udine is the nickname given to an unidentified killer. Most of the victims were prostitutes. The criminal was never identified, and the weapon was never found']

df_final.loc[df_final['Name'] == 'Tadeusz Grzesik', [
    'Nicknames', 'Conviction', 'Criminal_Penalty', 'Country',
    'Proven_victims', 'Possible_victims', 'Start_year', 'End_year',
    'Born_Date', 'Born_Location', 'Notes'
]] = [
    'Strawberry',                        # Nicknames
    'Murder',                            # Conviction
    'Life imprisonment',                 # Criminal_Penalty
    'Poland',                            # Country
    8,                                   # Proven_victims
    20,                                  # Possible_victims
    1991,                                # Start_year
    2007,                                # End_year
    '1960-01-01',                        # Born_Date
    'Wola Jachowa, Poland',              # Born_Location
    'A Polish strawberry grower and serial killer, known as "Strawberry" and "Killer". He led the Moneychangers Gang']

#verificamos
df_final[df_final['Name'] == 'Monster Of Udine']
df_final[df_final['Name'] == 'Tadeusz Grzesik']





In [ ]:
# Filtrar registros con Country nulo
nulos_country = df_final[df_final['Country'].isnull()]
display(nulos_country)



## Gestión de nulos

In [ ]:
#Ver nulos por columna
total = len(df_final)
nulos_df = pd.DataFrame({
    'Nulos': df_final.isnull().sum(),
    'Porcentaje': (df_final.isnull().sum() / total * 100).round(2)}).sort_values(by='Nulos', ascending=False)
print("Nulos y porcentaje por columna:")
print(nulos_df)


In [ ]:
#Posible_victims y Proven_victims son esenciales para el analisis por lo que pasamos a borrar los registros que no tienen esos datos.
registros_sin_victimas = df_final['Proven_victims'].isnull() & df_final['Possible_victims'].isnull()
df_final = df_final[~registros_sin_victimas].reset_index(drop=True)
#comprobamos viendo de nuevo los nulos.
filtro = df_final['Proven_victims'].isnull() | df_final['Possible_victims'].isnull()
df_final.loc[filtro, ['Name', 'Proven_victims', 'Possible_victims']]


In [ ]:
#Le paso nos nombre a chat gepete y le pido que busque informacion sobres las posibles victimas. 
#Tras la búsqueda decidimos que los asesinos que  no tenemos datos concisos se sustituya el nulo por 0 y en los casos que si tengamos datos se tomen esos datos para imputar los nulos.
df_final.loc[df_final['Name'] == 'José Paz Bezerra', ['Possible_victims']] = [24]
df_final.loc[df_final['Name'] == 'Rainbow Maniac', ['Possible_victims']] = [16]
df_final['Possible_victims'] = df_final['Possible_victims'].fillna(0)


In [ ]:
#Volvemos a revisar nulos por columna
total = len(df_final)
nulos_df = pd.DataFrame({
    'Nulos': df_final.isnull().sum(),
    'Porcentaje': (df_final.isnull().sum() / total * 100).round(2)}).sort_values(by='Nulos', ascending=False)
print("Nulos y porcentaje por columna:")
print(nulos_df)

In [ ]:
#columnas como la ubicacion de nacimiento y muerte, tiene un alto porcentaje de nulos y no las vemos útiles para el analisis. Decidimos borrarlas
df_final.drop(columns=['Died_Location', 'Born_Location'], inplace=True)

In [ ]:
# Filtrar registros con Years_active vacío
nulos_year = df_final[df_final['Years_active'].isnull()]
print(nulos_year.shape[0])
display(nulos_year)


In [ ]:
#5 registros de End_year y start_year son asumibles. Pido a chat gepete que busque la info para esos asesinos, e imputar asi esos datos.
df_final.loc[df_final['Name'] == 'Baba Anujka', 'End_year'] = 1928
df_final.loc[df_final['Name'] == 'John Duffy and David Mulcahy', 'End_year'] = 1986
df_final.loc[df_final['Name'] == 'Murder Incorporated', 'End_year'] = 1940
df_final.loc[df_final['Name'] == 'Abboud and Khajawa', 'Start_year'] = 1917

In [ ]:
#Combinando las columnas years_active, start_year y end_year podemos intentar completar los datos nulos de las mismas
# Rellenar Years_active si está vacío y hay Start y End
df_final['Years_active'] = df_final['Years_active'].fillna(df_final['End_year'] - df_final['Start_year'])

#Podemos seguir la misma estrategia con las columnas Age, Born_Date y Died_Date 
def calcular_edad(born, died):
    try:
        born = pd.to_datetime(born, errors='coerce')
        died = pd.to_datetime(died, errors='coerce')
        if pd.notnull(born) and pd.notnull(died):
            return died.year - born.year
        else:
            return None
    except:
        return None

df_final['Age_calculada'] = df_final.apply(lambda row: calcular_edad(row['Born_Date'], row['Died_Date']),axis=1)
df_final['Age'] = df_final['Age'].fillna(df_final['Age_calculada'])
df_final.drop(columns='Age_calculada', inplace=True)


In [ ]:
#Volvemos a revisar nulos por columna
total = len(df_final)
nulos_df = pd.DataFrame({
    'Nulos': df_final.isnull().sum(),
    'Porcentaje': (df_final.isnull().sum() / total * 100).round(2)}).sort_values(by='Nulos', ascending=False)
print("Nulos y porcentaje por columna:")
print(nulos_df)

In [ ]:
# Eliminar columnas que tienen muchos nulos
df_final.drop(columns=['Age', 'Died_Date', 'Born_Date', 'Date_Apprehended'], inplace=True)

In [ ]:
#reordenar columnas
column_order = ["Name", "Nicknames", "Country", "Proven_victims", "Possible_victims", "Years_active", "Start_year", "End_year", "Conviction", "Criminal_Penalty", "Notes"]
df_final = df_final[column_order]


Vemos que nuestro data set es un tanto escaso y decidimos completar con información haciendo un web scraping del sexo de los asesinos así como de las armas utilizadas.

In [ ]:
df_final.info()

In [ ]:
df_scrapeo1 = pd.read_csv('Primary-data/scrapeo1.csv')

In [ ]:
df_scrapeo1.info()

In [ ]:

df_final["Name"] = df_final["Name"].str.replace('"', '').str.strip().str.title()
df_scrapeo1["Name"] = df_scrapeo1["Name"].str.replace('"', '').str.strip().str.title()
df_completo = pd.merge(df_final, df_scrapeo1, on="Name", how="left")
print(df_completo.isna().sum())


In [ ]:
# Filtra los registros donde Weapon o Gender sean nulos
nulos_weapon_gender = df_completo[df_completo["Weapon"].isna() | df_completo["Gender"].isna()]

# Muestra resultados
print(f"Registros con NaN en Weapon o Gender: {nulos_weapon_gender.shape[0]}")
display(nulos_weapon_gender)


In [ ]:
# Lista de nombres a eliminar por falta de datos esenciales.
nombres_a_eliminar = [
    "John Justin Bunting, Robert Joe Wagner And James Vlassakis",
    "Satish",
    "Silvia Merez Cult"]

# Eliminamos del dataframe
df_completo = df_completo[~df_completo["Name"].isin(nombres_a_eliminar)]

# Confirmamos que ya no están
print(df_completo[df_completo["Name"].isin(nombres_a_eliminar)])


In [ ]:
#vamos a quitar los acentos que nos dan problemas.
def quitar_acentos(texto):
    if pd.isna(texto):
        return texto
    return unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')

df_completo['Name'] = df_completo['Name'].apply(quitar_acentos)


In [ ]:
#Vemos los valores unicos de weapon para poder establecer categorias para que sea más facil el analisis en tableau
valores_weapon = sorted(df_completo["Weapon"].dropna().unique())
for weapon in valores_weapon:
    print(weapon)


In [ ]:
def categorizar_weapon(valor):
    if pd.isna(valor):
        return "Unknown"
    valor = valor.lower()
    if any(p in valor for p in ["gun", "pistol", "revolver", "shotgun", "shooting", "shot"]):
        return "Firearm"
    elif any(p in valor for p in ["knife", "machete", "razor", "stabbed", "stabbing"]):
        return "Blade"
    elif any(p in valor for p in ["club", "bat", "hammer", "blunt object"]):
        return "Blunt Object"
    elif any(p in valor for p in ["strangled", "strangulation", "garrote", "choked", "smothered", "suffocated"]):
        return "Strangulation"
    elif any(p in valor for p in ["poison", "poisoned", "arsenic", "cyanide", "chemical", "chloroform"]):
        return "Poison"
    elif any(p in valor for p in ["car", "truck", "vehicle"]):
        return "Vehicle"
    elif any(p in valor for p in ["fire", "burned", "set on fire", "burnt", "explosives", "bomb", "grenade"]):
        return "Fire"
    elif any(p in valor for p in ["drowned", "drowning"]):
        return "Drowning"
    elif any(p in valor for p in ["beheaded", "decapitated"]):
        return "Beheading"
    elif any(p in valor for p in ["injected", "drugs", "medication"]):
        return "Drugs"
    elif any(p in valor for p in ["tortured", "beat to death", "incinerated", "disemboweled"]):
        return "Torture"
    elif any(p in valor for p in ["hanged", "hanging"]):
        return "Hanging"
    elif valor == "unknown":
        return "Unknown"
    else:
        return "Unknown" 


df_completo["Weapon_Category"] = df_completo["Weapon"].apply(categorizar_weapon)
print(df_completo["Weapon_Category"].value_counts())


In [ ]:
#Guardamos el CSV final 
os.makedirs("Final-data", exist_ok=True)
output_path = "Final-data/final_final.csv"
df_completo.to_csv(output_path, index=False)
print(f"Archivo guardado correctamente en: {output_path}")